In [1]:
import requests,re,time,numpy as np,pickle,asyncio,concurrent.futures

In [12]:
# Setup relevant variables
ZIP = '60615'
pickupDate = '10/20/2017'
dropoffDate = '10/27/2017'
TIME = '11:00 AM'
RADIUS = '40'

In [21]:
%run core_functions_debug.ipynb

In [14]:
results = runparser(threads=5)
print(results)

001 OK | 1 ('enterprise', 'E11586', 'Gg Hyde Park') ('0.95', '41.794955', '-87.587695')
006 ND | 6 ('enterprise', 'E115UG', 'Uber Southside') ('4.20', '41.811727', '-87.684696')
003 OK | 3 ('avis', 'CHIC05', 'Chicago - 18th Street & Wabash Avenue') ('4.05', '41.856873', '-87.62607')
005 OK | 5 ('enterprise', 'E115DN', 'Gg South Western') ('4.19', '41.811259', '-87.68454')
004 OK | 4 ('budget', 'CHIC02', 'Chicago') ('4.05', '41.856873', '-87.62607')
002 OK | 2 ('enterprise', 'E11576', 'Gg South Loop') ('3.17', '41.844252', '-87.623689')
007 OK | 7 ('budget', 'CHIC08', 'Chicago') ('5.11', '41.872643', '-87.62627')
008 OK | 8 ('enterprise', 'E11530', 'Gg Stony Island') ('5.17', '41.727246', '-87.585492')
010 OK | 10 ('avis', 'CHIX01', 'Chicago Union Train Station') ('5.71', '41.878674', '-87.640335')
009 OK | 9 ('enterprise', 'E115DA', 'Gg Gresham') ('5.57', '41.735849', '-87.66861')
011 OK | 11 ('budget', 'CHIX01', 'Chicago Union Train Station') ('5.71', '41.878674', '-87.640335')
012 OK

092 OK | 92 ('alamo', 'ORDT71', 'Chicago Ohare Airport') ('19.06', '41.979899', '-87.885468')
095 ND | 95 ('enterprise', 'E115P1', 'Packer Branch Elmhurst') ('19.34', '41.926378', '-87.939894')
094 OK | 94 ('enterprise', 'E11501', 'Aa Elmhurst') ('19.34', '41.92641', '-87.939792')
093 OK | 93 ('enterprise', 'E11508', 'Aa Westmont') ('19.16', '41.808409', '-87.9759')
096 OK | 96 ('enterprise', 'E11532', 'Gg Highland') ('19.43', '41.538019', '-87.471322')
097 OK | 97 ('enterprise', 'E115GR', 'Gg Griffith') ('19.55', '41.551021', '-87.427419')
098 OK | 98 ('enterprise', 'ORDT61', 'Ohare Intl Arpt') ('19.79', '41.995592', '-87.885729')
099 OK | 99 ('enterprise', 'E115AL', 'Aa Lombard') ('19.86', '41.86029', '-87.98148')
101 OK | 101 ('enterprise', 'E11550', 'Nn-park Ridge') ('20.19', '42.024864', '-87.855535')
100 OK | 100 ('enterprise', 'E115AK', 'Aa W Grand Ave') ('20.15', '41.9305', '-87.954659')
102 OK | 102 ('avis', 'CHIW10', 'Lombard') ('20.30', '41.86054', '-87.99008')
103 OK | 103 

182 ND | 182 ('enterprise', 'E115PA', 'Packer Branch Joliet') ('33.51', '41.5225', '-88.135302')
181 OK | 181 ('enterprise', 'E11545', 'Aa Palatine') ('32.84', '42.150901', '-88.035398')
183 OK | 183 ('enterprise', 'E11518', 'Aa Joliet') ('33.53', '41.522407', '-88.135687')
184 OK | 184 ('enterprise', 'E115PL', 'Aa Plainfield') ('35.69', '41.54539', '-88.2047')
185 OK | 185 ('avis', 'CHIW09', 'Foxfield Commons Shopping Plaza') ('35.80', '41.921238', '-88.280365')
186 OK | 186 ('budget', 'CHIW01', 'St.charles') ('35.80', '41.921238', '-88.280365')
187 OK | 187 ('enterprise', 'E115DS', 'Aa Shorewood') ('35.82', '41.523572', '-88.189941')
188 OK | 188 ('enterprise', 'E115AO', 'Aa Oswego') ('36.35', '41.710606', '-88.298573')
189 OK | 189 ('enterprise', 'E115N5', 'Nn-mundelein') ('36.51', '42.2392', '-87.9996')
190 OK | 190 ('enterprise', 'E11539', 'Gg Valparaiso') ('36.52', '41.46203', '-87.062748')
191 OK | 191 ('enterprise', 'E11542', 'Aa Aurora') ('36.65', '41.773929', '-88.314159')
19

In [28]:
addresses = []
for i in range(1,len(results)+1):
    entry = results[i]
    addresses.append(entry['addr']+', '+entry['city']+', '+entry['state']+', '+entry['zip']+', USA')
batchsize = 100
fullbatches = len(addresses)//batchsize
print(len(addresses))
destinations = []
for i in range(0,fullbatches):
    #print(i)
    destinations.append('|'.join(addresses[i*batchsize:(i+1)*batchsize]))
if len(addresses)%batchsize > 0:
    destinations.append('|'.join(addresses[fullbatches*batchsize:]))
print([destinations[i].count('|')+1 for i in range(0,len(destinations))])
origin = '1369 East Hyde Park Blvd, Chicago, IL, 60615, USA'

206
[100, 100, 6]


In [29]:
for i, dest in enumerate(destinations):
    gresponse = getGDistance(origin, dest)
    time.sleep(0.5)
    #print(gresponse)
    if gresponse['status'] == 'OK':        
        elems = gresponse['rows'][0]['elements']
        print('Got gmatrix response for batch {}|{}'.format(i,len(elems)))
        for j,entry in enumerate(elems):
            idx = i*batchsize+j+1
            if (entry['status'] == 'OK'):
                results[idx]['time'] = entry['duration']['value']
                results[idx]['roaddist'] = entry['distance']['value']
                #print(idx,results[idx]['time'],results[idx]['roaddist'])
            else:
                print('{} ENTRY FAILED'.format(idx))
    else:
        print('Gmatrix response for batch {} FAILED'.format(i))

Got gmatrix response for batch 0|100
96 ENTRY FAILED
97 ENTRY FAILED
Got gmatrix response for batch 1|100
105 ENTRY FAILED
115 ENTRY FAILED
116 ENTRY FAILED
126 ENTRY FAILED
133 ENTRY FAILED
134 ENTRY FAILED
138 ENTRY FAILED
149 ENTRY FAILED
152 ENTRY FAILED
159 ENTRY FAILED
160 ENTRY FAILED
178 ENTRY FAILED
185 ENTRY FAILED
186 ENTRY FAILED
187 ENTRY FAILED
188 ENTRY FAILED
189 ENTRY FAILED
190 ENTRY FAILED
192 ENTRY FAILED
197 ENTRY FAILED
198 ENTRY FAILED
Got gmatrix response for batch 2|6
203 ENTRY FAILED
205 ENTRY FAILED


In [24]:
car1 = 'Economy Car'
car2 = 'Compact Car'
car3 = 'Intermediate Car'
car4 = 'Intermediate SUV'
desiredcar = car2
desiredsort = 'time'
desiredbrand = 'enterprise'

# also, can optimize into some single query...buuuut it works as is
rvals = np.array([x for x in results.values() if desiredcar in x and desiredsort in x and x['brand'] == desiredbrand])
rvals = np.array([x for x in results.values() if desiredcar in x and desiredsort in x])
ordertime = np.array([x[desiredsort] for x in rvals]).argsort()
orderprice = np.array([x[desiredcar] for x in rvals]).argsort()
order = [x for x in ordertime if x in orderprice[:70]]

In [25]:
orderedresults = rvals[order]
print(data['pickupDate'],data['pickupTime'],data['dropoffDate'],data['dropoffTime'],desiredcar,car2,car3)
for k in orderedresults:
    #print(k['num'])
    print('{:03d} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
          .format(k['num'],float(k['dist']),k['time']//60,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))

10/20/2017 11:00 AM 10/27/2017 11:00 AM Compact Car Compact Car Intermediate Car
050 | 10.82 |  75 | 166.27 | 166.27 | 166.37 | budget     | Oak Park
047 | 10.78 |  82 | 158.02 | 158.02 | 159.51 | budget     | Chicago Ridge Il Sears
074 | 16.99 |  84 | 158.02 | 158.02 | 159.51 | budget     | Orland Park
033 |  8.04 |  84 | 158.02 | 158.02 | 159.51 | budget     | Oak Lawn
112 | 21.35 |  85 | 142.18 | 142.18 | 144.64 | avis       | Downers Grove, Il
113 | 21.35 |  85 | 135.72 | 135.72 | 139.10 | budget     | Downers Grove Il
114 | 21.41 |  87 | 157.51 | 157.51 | 167.05 | enterprise | Aa North Downers Grove
077 | 17.63 |  87 | 162.33 | 162.33 | 178.45 | enterprise | Schiller Park
066 | 15.03 |  93 | 158.02 | 158.02 | 159.51 | budget     | Lincolnwood
141 | 27.09 |  96 | 150.34 | 150.34 | 159.88 | enterprise | Aa East Naperville
130 | 24.83 |  96 | 150.34 | 150.34 | 159.88 | enterprise | Aa Lisle
085 | 18.35 |  97 | 158.02 | 158.02 | 159.51 | budget     | Skokie
140 | 27.04 |  97 | 142.18 

###### brands = np.array([x['brand'] for x in results2.values()])[order]
nums = np.array([x for x in results2.keys()])[order]
titles = np.array([x['name'] for x in results2.values()])[order]
prices = np.array([x[desiredcar] for x in results2.values()])[order]
prices2 = np.array([x[car2] for x in results2.values()])[order]
prices3 = np.array([x[car3] for x in results2.values()])[order]
distances = np.array([x['dist'] for x in results2.values()])[order]

print(data['pickupDate'],data['pickupTime'],data['dropoffDate'],data['dropoffTime'],desiredcar,car2,car3)
for k in zip(nums,prices,prices2,prices3,brands,titles,distances):
    #print(k)
    print('{:03d} | {:5.2f} | {:5.2f} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'.format(k[0],float(k[6]),k[1],k[2],k[3],str(k[4]),k[5]))